In [74]:
import numpy as np
import math
import matplotlib.pyplot as plt
import xlrd

In [75]:
def load_dataset(file_name):
    data = xlrd.open_workbook(file_name)
    table = data.sheets()[0]
    nrows = table.nrows # 行数
    ncols = table.ncols # 列数
    datamatrix = np.zeros((nrows-1, ncols-2))
    for x in range(1, ncols-1):
        cols = table.col_values(x)
        cols.remove(cols[0])
        cols1 = np.matrix(cols)  # 把list转换为矩阵进行矩阵操作
        datamatrix[:, x-1] = cols1 # 把数据进行存储
    return np.transpose(datamatrix)

In [76]:
data_set = load_dataset(r"D:\学校\项目研究进展\模拟数据.xls")
print(data_set)

[[1.48365051e+02 1.49507629e+02 1.49508240e+02 ... 1.70310730e+02
  1.70283386e+02 1.51642761e+02]
 [1.94553676e+01 1.43883495e+01 1.43814924e+01 ... 1.58973204e+01
  1.58989562e+01 5.42989591e+00]
 [5.47541360e-01 5.49905934e-01 8.67773210e-01 ... 6.25374588e-01
  8.83968099e-01 8.50978254e-01]
 ...
 [8.98193359e-01 9.05517578e-01 1.29150391e-01 ... 9.53857422e-01
  1.75292969e-01 1.43066406e-01]
 [1.38452027e+00 1.34621615e+00 1.35354388e+00 ... 1.34413248e+00
  1.33085350e+00 1.29445434e+00]
 [9.12554462e-01 9.07904299e-01 9.19799805e-01 ... 8.78489323e-01
  8.96728516e-01 9.12841797e-01]]


In [77]:
# 二分K均值聚类的主函数，输入参数为数据集和要分的簇的个数
def bi_Kmeans(data_set, k):   
    m = data_set.shape[0]   # 得到data_set的行数，即数据集的个数
    cluster_assment = np.zeros((m,2)) # 初始化样本点的簇分配矩阵，第一列为簇分配索引，第二列为欧氏距离平方
    centroid0 = np.mean(data_set, axis=0)  # 按列计算均值，即找到初始质心
    cent_list = [centroid0]
    for j in range(m):   # 对于每个样本点
        cluster_assment[j,1] = dist_eclud(centroid0, data_set[j,:])**2 # 计算该样本点的误差平方
    while (len(cent_list) < k):   # 当已有的簇个数小于k时，迭代执行以下代码
        lowestSSE = np.inf     # 初始化误差平方和SSE的最小值

        # 找到对哪个簇进行划分可以最大程度降低SSE值
        for i in range(len(cent_list)):   # 遍历每个已有的簇
            # 得到属于该簇的所有样本点
            pts_incurr_cluster = \
            data_set[np.nonzero(list(map(lambda x:x==i, cluster_assment[:,0])))]
            # 将该簇的所有样本点通过函数Kmean进行划分(k=2),得到划分后的质心和簇分配矩阵
            centroid_mat, split_clust_ass = Kmeans(pts_incurr_cluster, 2)    
            sse_split = np.sum(split_clust_ass[:,1])  # 得到划分后的误差平方和
            # 得到其他样本点的误差平方和
            sse_not_split = \
            np.sum(cluster_assment[np.nonzero(list(map(lambda x:x!=i, cluster_assment[:,0]))),1])
            if (sse_split + sse_not_split) < lowestSSE:  # 如果总的误差平方和小于lowestSSE
                best_cent_to_split = i                   # 则保存本次划分
                best_new_cents = centroid_mat
                best_clust_ass = split_clust_ass
                lowestSSE = sse_split + sse_not_split
        # 对最大程度降低SSE值的簇进行划分

        # 将划分后得到的编号为0的结果簇的编号修改为原最大簇编号+1，即len(cent_list)
        best_clust_ass[np.nonzero(list(map(lambda x:x==1, best_clust_ass[:,0]))), 0] = len(cent_list)  
         # 将划分后得到的编号为1的结果簇的编号修改为被划分的簇的编号
        best_clust_ass[np.nonzero(list(map(lambda x:x==0, best_clust_ass[:,0]))), 0] = best_cent_to_split 
        cent_list[best_cent_to_split] = best_new_cents[0,:] # 更新被划分的簇的质心
        cent_list.append(best_new_cents[1,:])  # 添加一个新的簇的质心
        cluster_assment[np.nonzero(list(map(lambda x:x==best_cent_to_split, cluster_assment[:,0]))),:] = \
            best_clust_ass  # 将簇分配矩阵中属于被划分的簇的样本点的簇分配进行更新
    return np.array(cent_list), cluster_assment

In [78]:
cent_list, cluster_assment = bi_Kmeans(data_set, 2)
print(cent_list)
print(cluster_assment)

[[  2.0578576    1.55928113   1.5353981  ...   1.66157848   1.65015098
    0.92371896]
 [148.75863987 149.70539426 149.71909291 ... 170.5334959  170.53615748
  151.63131593]]
[[1.00000000e+00 4.66306499e+01]
 [0.00000000e+00 1.30988442e+05]
 [0.00000000e+00 8.38920132e+02]
 [0.00000000e+00 1.85969576e+03]
 [0.00000000e+00 1.84248365e+03]
 [0.00000000e+00 1.81277974e+03]
 [0.00000000e+00 1.72210822e+03]
 [0.00000000e+00 1.72841796e+03]
 [0.00000000e+00 1.85541564e+03]
 [0.00000000e+00 1.82532609e+03]
 [0.00000000e+00 8.36850169e+02]
 [0.00000000e+00 1.85970725e+03]
 [0.00000000e+00 1.85638748e+03]
 [0.00000000e+00 1.82623905e+03]
 [0.00000000e+00 1.72794031e+03]
 [0.00000000e+00 1.73273379e+03]
 [0.00000000e+00 1.84248755e+03]
 [0.00000000e+00 1.81282308e+03]
 [1.00000000e+00 2.91441562e+00]
 [0.00000000e+00 2.31010764e+04]
 [0.00000000e+00 3.24690891e+02]
 [0.00000000e+00 4.49972251e+02]
 [0.00000000e+00 3.06962740e+02]
 [0.00000000e+00 4.20165817e+02]
 [0.00000000e+00 1.22541927e+03]
